<a href="https://colab.research.google.com/github/anushka-code/Feature_Envy_Detection_Using_Deep_Learning/blob/main/Feature_Envy_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading the Dataset from Google Drive

In [8]:
!pip install -U -q PyDrive

In [9]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
  
  
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [11]:
link = 'https://drive.google.com/file/d/1bnQ_b524o2XqHSaILYhqzec39ZpGJ_Kc/view?usp=sharing'

import pandas as pd
  
# to get the id part of the file
id = link.split("/")[-2]
  
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('feature_envy_classification.csv')  
  
df = pd.read_csv('feature_envy_classification.csv')
df.head(5)

,id,NOP_method,CC_method,ATFD_method,FDP_method,CM_method,MAXNESTING_method,LOC_method,CYCLO_method,NMCS_method,NOLV_method,MaMCL_method,NOAV_method,LAA_method,FANOUT_method,CFNAMM_method,ATLD_method,CLNAMM_method,CINT_method,MeMCL_method,CDISP_method,NOII_type,NOAM_type,NOCS_type,NOM_type,NMO_type,ATFD_type,FANOUT_type,NOMNAMM_type,NOA_type,NIM_type,DIT_type,LOC_type,LOCNAMM_type,CFNAMM_type,TCC_type,NOPA_type,CBO_type,RFC_type,NOC_type,...,AMW_type,AMWNAMM_type,NOCS_package,NOMNAMM_package,NOI_package,LOC_package,NOM_package,NOPK_project,NOCS_project,NOI_project,NOM_project,NOMNAMM_project,LOC_project,isStatic_type,number_private_visibility_attributes,number_protected_visibility_attributes,number_package_visibility_attributes,num_final_attributes,num_static_attributes,num_final_static_attributes,num_not_final_not_static_attributes,num_final_not_static_attributes,num_static_not_final_attributes,number_public_visibility_methods,number_private_visibility_methods,number_protected_visibility_methods,number_package_visibility_methods,number_final_methods,number_abstract_methods,number_not_abstract_not_final_methods,number_static_methods,number_final_static_methods,number_final_not_static_methods,number_not_final_static_methods,number_not_final_not_static_methods,number_standard_design_methods,number_constructor_DefaultConstructor_methods,number_constructor_NotDefaultConstructor_methods,isStatic_method,is_feature_envy
0,1,0,0,0,0,0,1,3,1,0,0,0,1,1.0,1,1,1,1,1,0,1.0,0,0,0,6,0,3,1,6,2,32,3,54,54,2,1.0,2,5,56,0,...,1.5,1.5,20,72,0,701,72,2,60,0,254,243,2453,0,0,0,0,0,0,0,2,0,0,2,4,0,0,0,0,6,0,0,0,0,6,6,0,0,0,False
1,2,0,0,0,0,0,1,3,1,0,0,0,1,1.0,1,1,1,1,1,0,1.0,0,0,0,6,0,3,1,6,2,32,3,54,54,2,1.0,2,5,56,0,...,1.5,1.5,20,72,0,701,72,2,60,0,254,243,2453,0,0,0,0,0,0,0,2,0,0,2,4,0,0,0,0,6,0,0,0,0,6,6,0,0,0,False
2,3,0,0,0,0,0,1,3,1,0,0,0,1,1.0,1,1,1,1,1,0,1.0,0,0,0,6,0,3,1,6,2,32,3,54,54,2,1.0,2,5,56,0,...,1.5,1.5,20,72,0,701,72,2,60,0,254,243,2453,0,0,0,0,0,0,0,2,0,0,2,4,0,0,0,0,6,0,0,0,0,6,6,0,0,0,False
3,4,0,0,0,0,0,1,3,1,0,0,0,1,1.0,1,1,1,1,1,0,1.0,0,0,0,6,0,3,1,6,2,32,3,54,54,2,1.0,2,5,56,0,...,1.5,1.5,20,72,0,701,72,2,60,0,254,243,2453,0,0,0,0,0,0,0,2,0,0,2,4,0,0,0,0,6,0,0,0,0,6,6,0,0,0,False
4,5,1,0,0,0,0,1,5,1,0,2,0,2,0.0,1,1,0,0,1,0,1.0,0,0,0,6,0,3,1,6,2,32,3,54,54,2,1.0,2,5,56,0,...,1.5,1.5,20,72,0,701,72,2,60,0,254,243,2453,0,0,0,0,0,0,0,2,0,0,2,4,0,0,0,0,6,0,0,0,0,6,6,0,0,0,False
